In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

import requests
import json

import string
import math

from collections import Counter
import random


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /cfs/home/u021320/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#test set 1 = season 8

with open('sets/test_set1.json') as f:
    test_set1 = json.load(f)

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

#CHANGE HERE
test_set = test_set1
train_set = train_set1

all_lines = []

#corpus
for line in train_set:
    all_lines.append(line['transcript'])

In [3]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit(all_lines)

feature_names = vectorizer.get_feature_names_out()


#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])


    tf_idf = vectorizer.transform(character_lines)

    dense = tf_idf.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    values = []

    for sentence in dense:
        values.append(sentence.sum())

    
    #associate values with sentences
    sentences = []
    for i in range(len(character_lines)):
        sentences.append({'sentence': character_lines[i], 'value': values[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    #get highest x sentences
    x = 10
    highest = sentences[:x]

    print(character)

    for sentence in highest:
        print(sentence)


Monica Geller
{'sentence': "Y'know, so I don't read as many important books as you do, and I don't write trick poems that seem to be about one thing but are actually about something else. And y'know what, I get excited about stupid stuff, like when I my People magazine comes on Saturday, and the new Hold Everything catalog. Y'know but that does not mean that I'm empty, I care about things. I care about my friends and family. You have no right to make that kind of judgment about me.", 'value': 6.907173678589953}
{'sentence': "Absolutely! Now get out of here you!! Okay, I'm running out of places I can touch him! Look, is there something wrong with me? I mean why am I only attracted to guys where there's no future? Either they're too old, or they're too young, and then there's Pete who's-who's crazy about me, and who's absolutely perfect for me, and there's like zip going on! I mean, seriously, does it sound like something's wrong with me?!", 'value': 6.616048819305871}
{'sentence': "No, 

In [6]:
def tf_idf_calculator(line, character_lines, idf):

    tf = character_lines.count(line) / len(character_lines)

    tf_idf = tf * idf[line]

    return tf_idf

In [ ]:
#all_lines = random.sample(all_lines, 200)

idf = {}

all_lines_set = [sentence.translate(str.maketrans('', '', string.punctuation)).lower() for sentence in all_lines]

for line in all_lines_set:
    count = all_lines_set.count(line)
    idf[line] = math.log(len(all_lines_set) / (1 + count))

#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])

    print(character)

    #character_lines = random.sample(character_lines, 10)
    
    tf_idfs = []

    character_lines_processed = []

    for line in character_lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).lower()
        character_lines_processed.append(line)

    for line in character_lines_processed:
        tf_idfs.append(tf_idf_calculator(line, character_lines_processed, idf))


    #associate values with sentences
    sentences = []
    for i in range(len(character_lines_processed)):
        sentences.append({'sentence': character_lines_processed[i], 'value': tf_idfs[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    sentences_only = [sentence['sentence'] for sentence in sentences]

    #remove duplicates
    sentences_only = list( dict.fromkeys(sentences_only) )

    #get highest x sentences
    x = 15
    highest = sentences_only[:x]

    for sentence in highest:
        print(sentence)

    print("\n")

In [8]:
#all_lines = random.sample(all_lines, 200)

############ STOP WORDS ############

idf = {}

all_lines_set = [sentence.translate(str.maketrans('', '', string.punctuation)).lower() for sentence in all_lines]

all_lines_set_no_stopwords = []

for line in all_lines_set:
    if line not in stopwords.words('english'):
        all_lines_set_no_stopwords.append(line)


for line in all_lines_set_no_stopwords:
    count = all_lines_set_no_stopwords.count(line)
    idf[line] = math.log(len(all_lines_set_no_stopwords) / (1 + count))

#idf = calculate_custom_idf(all_lines_set_no_stopwords)

#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])

    print(character)

    #character_lines = random.sample(character_lines, 10)
    
    tf_idfs = []

    character_lines_processed = []

    for line in character_lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).lower()
        if line not in stopwords.words('english'):
            character_lines_processed.append(line)

    for line in character_lines_processed:
        tf_idfs.append(tf_idf_calculator(line, character_lines_processed, idf))


    #associate values with sentences
    sentences = []
    for i in range(len(character_lines_processed)):
        sentences.append({'sentence': character_lines_processed[i], 'value': tf_idfs[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    sentences_only = [sentence['sentence'] for sentence in sentences]

    #remove duplicates
    sentences_only = list( dict.fromkeys(sentences_only) )

    #get highest x sentences
    x = 15
    highest = sentences_only[:x]

    for sentence in highest:
        print(sentence)

    print("\n")

Monica Geller
hey
okay
really
hi
yeah
oh my god
yes
oh
i know
chandler
bye
phoebe
thank you
joey
what are you doing


Joey Tribbiani
hey
yeah
okay
all right
hi
really
yes
oh yeah
huh
hello
i dont know
i know
oh
thank you
oh my god


Chandler Bing
hey
okay
yeah
oh my god
yes
really
all right
hi
i know
thanks
oh yeah
ok
hello
oh
bye


Phoebe Buffay
hey
yeah
okay
oh
hi
oh my god
uhhuh
really
sure
i know
huh
bye
thanks
oh no
ooh


Ross Geller
yeah
hey
hi
okay
oh
i know
thank you
hello
huh
really
hey guys
all right
thanks
oh my god
ohh


Rachel Green
yeah
okay
hi
hey
oh
oh my god
really
yes
right
uhhuh
i know
all right
thank you
ok
oh hi


others
hi
hey
yeah
okay
oh
yes
bye
really
oh my god
hello
sure
wow

rachel
great




In [4]:
def tf_idf_calculator2(line, character_lines, idf):

    counts = 0

    for sentence in character_lines:
        if line in sentence:
            counts += 1

    tf = counts / len(character_lines)

    tf_idf = tf * idf[line]
    
    return tf_idf

In [9]:
def calculate_custom_idf2(all_lines):
    idf = {}
    total_documents = len(all_lines)

    print("Total documents:", total_documents)
    print("fraction ", total_documents * 0.007)

    term_document_count = {}
    for line in all_lines:
        counts = 0
        for sentence in all_lines_set:
            if line in sentence:
                counts += 1
        term_document_count[line] = counts

    # Compute the IDF values with customization
    for line, count in term_document_count.items():

        idf[line] = math.log(total_documents / (count + 1))  # Adding 1 to avoid division by zero

        # Customization: Lower the IDF for frequently occurring terms
        if count > total_documents * 0.007:  # Adjust the threshold as needed
            print("Lowering IDF for", line)
            idf[line] *= 0.05  # Adjust the penalty factor as needed

    return idf

In [10]:
idf = {}

all_lines_set = [sentence.translate(str.maketrans('', '', string.punctuation)).lower() for sentence in all_lines]

all_lines_set_no_stopwords = []

for line in all_lines_set:
    if line not in stopwords.words('english'):
        all_lines_set_no_stopwords.append(line)


for line in all_lines_set:
    counts = 0
    for sentence in all_lines_set:
        if line in sentence:
            counts += 1
    
    idf[line] = math.log(len(all_lines) / (1 + counts))

idf = calculate_custom_idf2(all_lines_set_no_stopwords)


#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])

    print(character)
    
    tf_idfs = []

    character_lines_processed = []

    for line in character_lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).lower()
        if line not in stopwords.words('english'):
            character_lines_processed.append(line)

    for line in character_lines_processed:
        tf_idfs.append(tf_idf_calculator2(line, character_lines_processed, idf))


    #associate values with sentences
    sentences = []
    for i in range(len(character_lines_processed)):
        sentences.append({'sentence': character_lines_processed[i], 'value': tf_idfs[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    sentences_only = [sentence['sentence'] for sentence in sentences]

    #remove duplicates
    sentences_only = list( dict.fromkeys(sentences_only) )

    #get highest x sentences
    x = 15
    highest = sentences_only[:x]

    for sentence in highest:
        print(sentence)

    print("\n")

Total documents: 42535
fraction  297.745
Lowering IDF for hi
Lowering IDF for ohh
Lowering IDF for thanks
Lowering IDF for sorry
Lowering IDF for rachel
Lowering IDF for yes
Lowering IDF for really
Lowering IDF for yeah
Lowering IDF for you guys
Lowering IDF for oh my god
Lowering IDF for stop
Lowering IDF for oh yeah
Lowering IDF for thank you
Lowering IDF for okay
Lowering IDF for i did
Lowering IDF for no no
Lowering IDF for mm
Lowering IDF for ross
Lowering IDF for huh
Lowering IDF for wow
Lowering IDF for hello
Lowering IDF for i know
Lowering IDF for hey
Lowering IDF for well
Lowering IDF for im sorry
Lowering IDF for pheebs
Lowering IDF for phoebe
Lowering IDF for oh no
Lowering IDF for joey
Lowering IDF for sure
Lowering IDF for oh
Lowering IDF for ow
Lowering IDF for all right
Lowering IDF for chandler
Lowering IDF for night
Lowering IDF for come on
Lowering IDF for ok
Lowering IDF for you dont
Lowering IDF for you do
Lowering IDF for i do
Lowering IDF for see
Lowering IDF for